In [ ]:
from transformers import VideoMAEForVideoClassification
from transformers import AutoImageProcessor, VideoMAEForPreTraining, VideoMAEConfig
import torch.nn as nn
import torch
import numpy as np


In [ ]:
model_ckpt = "MCG-NJU/videomae-base" # pre-trained model from which to fine-tune
model = VideoMAEForVideoClassification.from_pretrained(model_ckpt, attn_implementation="sdpa", torch_dtype=torch.float16)

In [ ]:
print(model.classifier.in_features)
print(model.classifier.out_features)

In [ ]:
config = VideoMAEConfig()
custom_classes_classifier = VideoMAEForVideoClassification(config)
custom_classes_classifier.config

In [ ]:
num_frames = 16
video = list(np.random.randint(0, 256, (num_frames, 3, 224, 224)))

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape)

In [ ]:
model.classifier = nn.Linear(model.classifier.in_features, 10)
#random initialize weights
nn.init.xavier_uniform_(model.classifier.weight)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
pixel_values = image_processor(video, return_tensors="pt", use_fast = True).pixel_values


In [ ]:
type(pixel_values)

In [ ]:
pixel_values.shape

In [ ]:
outputs = custom_classes_classifier(pixel_values)

In [ ]:
type(outputs)

In [ ]:
outputs.logits

In [ ]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

In [ ]:
from huggingface_hub import hf_hub_download

hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
destination_path = "/share/j_sun/jth264"  # Change this to your desired directory

file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, 
    filename=filename, 
    repo_type="dataset",
    local_dir=destination_path  # Specify the download directory
)


In [ ]:

video_path = '/share/j_sun/jth264/UCF101_subset/test/Basketball/v_Basketball_g02_c04.avi' 
container = av.open(video_path)

In [ ]:
container.streams.video[0].frames

In [ ]:
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

In [ ]:
video.shape

In [ ]:
torch.tensor(video).shape

In [ ]:
#rearrage the dimensions
video = torch.tensor(video).permute(0, 3, 1, 2)

In [ ]:
video.shape


In [ ]:
data_path = '/share/j_sun/jth264/UCF101_subset'
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
video_transform = lambda video: image_processor(list(video), return_tensors="pt").pixel_values
dataset = UCF101(data_path, transform=video_transform)

In [ ]:
import av
from fish_benchmark.utils import read_video_pyav
from transformers import AutoImageProcessor
clip_length = 16
video_path = '/share/j_sun/jth264/UCF101_subset/test/Basketball/v_Basketball_g02_c04.avi'
container = av.open(video_path)
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base", use_fast = True)
video_transform = lambda video: image_processor(list(video), return_tensors="pt")
total_frames = container.streams.video[0].frames
print(f"Total frames in the video: {total_frames}")
video = read_video_pyav(container, list(range(0,clip_length)))
type(video)
print(video.shape)
inputs = video_transform(video)

In [ ]:
from transformers import VideoMAEModel
model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

In [ ]:
output = model(**inputs)


In [ ]:
output.last_hidden_state.shape